In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import models, datasets, transforms
from PIL import Image
import os
import zipfile
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

# Load experimental data
def load_all_experimental_data(test_digits_folder):
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []
    participant_data = {}

    transform = transforms.Compose([
        transforms.Resize((16, 16)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    for filename in os.listdir(test_digits_folder):
        if filename.endswith('.zip') and filename.startswith('experiment_results_participant'):
            participant_number = int(filename.split('participant')[1].split('.')[0])
            zip_filepath = os.path.join(test_digits_folder, filename)

            participant_train_images = []
            participant_train_labels = []
            participant_test_images = []
            participant_test_labels = []

            with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                for img_filename in zip_ref.namelist():
                    if img_filename.endswith('.png'):
                        with zip_ref.open(img_filename) as file:
                            img = Image.open(file).convert('L')
                            img_tensor = transform(img)
                            
                            digit = int(img_filename.split('_')[0])
                            
                            if 'composite' in img_filename:
                                test_images.append(img_tensor)
                                test_labels.append(digit)
                                participant_test_images.append(img_tensor)
                                participant_test_labels.append(digit)
                            else:
                                train_images.append(img_tensor)
                                train_labels.append(digit)
                                participant_train_images.append(img_tensor)
                                participant_train_labels.append(digit)

            participant_data[participant_number] = {
                'train': (torch.stack(participant_train_images), torch.tensor(participant_train_labels)),
                'test': (torch.stack(participant_test_images), torch.tensor(participant_test_labels))
            }

    return (torch.stack(train_images), torch.tensor(train_labels), 
            torch.stack(test_images), torch.tensor(test_labels),
            participant_data)

# Load experimental data
exp_train_images, exp_train_labels, exp_test_images, exp_test_labels, participant_data = load_all_experimental_data('test_digits')

In [2]:
from sklearn.model_selection import train_test_split

# Split the test data into validation and test sets
val_images, final_test_images, val_labels, final_test_labels = train_test_split(
    exp_test_images, exp_test_labels, test_size=0.5, random_state=42
)

# Assuming exp_train_images and exp_train_labels are already loaded
train_dataset = TensorDataset(exp_train_images, exp_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create DataLoader for validation set
val_dataset = torch.utils.data.TensorDataset(val_images, val_labels)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Create DataLoader for final test set
test_dataset = torch.utils.data.TensorDataset(final_test_images, final_test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
import torch.nn as nn

# Define LeNet5 architecture modified for 16x16 images
class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(16 * 1 * 1, 120)  # Adjusted for 16x16 input size
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Load LeNet5 model for 16x16 images
def load_lenet5_model_16x16():
    model = LeNet5_16x16(num_classes=10)
    return model

In [4]:
# Fine-tuning function for LeNet5 on your dataset
def fine_tune_lenet(model, train_loader, val_loader=None, num_epochs=15, save_path="lenet5_model.pth"):
    criterion = nn.CrossEntropyLoss()  # Loss function for classification tasks

    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

    model.train()  # Set model to training mode
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        # Training loop
        for images, labels in train_loader:
            optimizer.zero_grad()  # Zero the parameter gradients
            
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            
            loss.backward()  # Backward pass (compute gradients)
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}')
        
        # Validation loop (if validation set is provided)
        if val_loader:
            correct_val = 0
            total_val = 0
            model.eval()  # Set model to evaluation mode during validation
            
            with torch.no_grad():  # Disable gradient computation during validation
                for val_images_batch, val_labels_batch in val_loader:
                    outputs_val = model(val_images_batch)
                    _, predicted_val = torch.max(outputs_val.data, 1)   # Get predicted class
                    
                    total_val += val_labels_batch.size(0)
                    correct_val += (predicted_val == val_labels_batch).sum().item()
            
            print(f'Validation Accuracy: {100 * correct_val / total_val:.2f}%')
            
            model.train()  # Switch back to training mode after validation

    # Save the trained model after training is complete
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

# Example usage:
lenet_model_16x16 = load_lenet5_model_16x16()
fine_tune_lenet(lenet_model_16x16, train_loader=train_loader, val_loader=val_loader, save_path="lenet5_trained_model.pth")

Epoch [1/15], Loss: 2408.0627
Validation Accuracy: 68.42%
Epoch [2/15], Loss: 2377.3934
Validation Accuracy: 73.68%
Epoch [3/15], Loss: 2365.0198
Validation Accuracy: 75.79%
Epoch [4/15], Loss: 2358.5327
Validation Accuracy: 85.26%
Epoch [5/15], Loss: 2354.6844
Validation Accuracy: 90.53%
Epoch [6/15], Loss: 2351.0160
Validation Accuracy: 90.53%
Epoch [7/15], Loss: 2346.9722
Validation Accuracy: 74.74%
Epoch [8/15], Loss: 2344.3531
Validation Accuracy: 86.32%
Epoch [9/15], Loss: 2342.2535
Validation Accuracy: 88.42%
Epoch [10/15], Loss: 2341.1659
Validation Accuracy: 96.84%
Epoch [11/15], Loss: 2339.2536
Validation Accuracy: 94.74%
Epoch [12/15], Loss: 2337.6042
Validation Accuracy: 95.79%
Epoch [13/15], Loss: 2336.2158
Validation Accuracy: 89.47%
Epoch [14/15], Loss: 2334.8472
Validation Accuracy: 88.42%
Epoch [15/15], Loss: 2334.1785
Validation Accuracy: 93.68%
Model saved to lenet5_trained_model.pth


In [6]:
def evaluate_lenet(model, test_loader):
    correct_test = 0
    total_test = 0
    
    model.eval()  # Set model to evaluation mode
    
    with torch.no_grad():  # Disable gradient computation for inference
        for images_batch_test, labels_batch_test in test_loader:
            outputs_test = model(images_batch_test)
            _, predicted_test_classifications = torch.max(outputs_test.data, 1)   # Get predicted class
            
            total_test += labels_batch_test.size(0)
            correct_test += (predicted_test_classifications == labels_batch_test).sum().item()
    
    print(f'Accuracy of the network on final test images: {100 * correct_test / total_test:.2f}%')

# Evaluate on final test set after training is complete
evaluate_lenet(lenet_model_16x16, test_loader=test_loader)

Accuracy of the network on final test images: 91.58%
